### Day 18

In [1]:
import numpy as np
from collections import defaultdict, Counter

In [2]:
day_i = 18

In [3]:
%run start_day.py $day_i

Initializing day 18


In [4]:
cd /home/vincent/Documents/AdventOfCode/2021

/home/vincent/Documents/AdventOfCode/2021


In [5]:
PATH = f"day{day_i}/input{day_i}"

In [6]:
!wc -l $PATH

100 day18/input18


In [7]:
!head $PATH

[5,[5,0]]
[[[[8,4],3],[6,7]],[[[7,4],2],[[5,7],[1,5]]]]
[[[[7,6],[4,6]],1],[[5,7],[0,[2,1]]]]
[[[[5,8],[3,5]],[9,0]],[[[4,0],[4,1]],[[9,3],1]]]
[7,[[[6,4],[2,7]],[[8,8],5]]]
[[9,6],[1,[[4,5],3]]]
[[[7,1],[2,4]],[[[6,2],[1,6]],[8,8]]]
[[[[1,9],[1,1]],[2,9]],5]
[7,[[[0,0],8],[[5,2],[7,6]]]]
[[[8,[8,7]],6],[7,[6,2]]]


In [8]:
!tail $PATH

[[[3,0],8],[2,[3,7]]]
[3,[[[7,0],5],[[2,2],[8,9]]]]
[[9,[4,6]],[[[3,6],8],[5,0]]]
[[[9,5],[[8,7],[0,0]]],[[8,[8,9]],[6,[8,1]]]]
[[[4,5],[[1,7],[9,1]]],[[[5,7],[8,9]],[[4,8],5]]]
[[[[0,7],8],[8,0]],[2,[[5,4],2]]]
[[[8,4],6],[3,[[0,7],[8,0]]]]
[[[[1,4],[3,7]],[[6,9],0]],[5,5]]
[[3,[[6,1],[8,7]]],[[7,8],9]]
[[[[8,5],[7,5]],[[1,0],[2,7]]],[1,[[2,3],[2,8]]]]


In [9]:
def parse_input(inp):
    return [eval(x) for x in inp]

In [10]:
# real input
with open(PATH, 'r') as f:
    inputs = parse_input([x.strip() for x in f.readlines()])


In [92]:
# test input
test_str1 = """[1,1]
[2,2]
[3,3]
[4,4]
[5,5]
[6,6]"""

test_str2 = """[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]"""

inputs_1 = parse_input(test_str1.split('\n'))
inputs_2 = parse_input(test_str2.split('\n'))

In [154]:
class Number():
    def __init__(self, l):
        self.content = l
        self.type = "integer" if type(l) == int else "list"
        
    def __add__(self, o):
        self.content = [self.content, o.content]
        self.do_reduce()
        return self
    
    def __str__(self):
        return str(self.content)
    
    def add_to_left(self, n):
        if n is None:
            return self.content
        if self.type == "integer":
            return self.content + n
        else:
            return [Number(self.content[0]).add_to_left(n), self.content[1]]
    
    def add_to_right(self, n):
        if n is None:
            return self.content
        if self.type == "integer":
            return self.content + n
        else:
            return [self.content[0], Number(self.content[1]).add_to_right(n)]
    
    def do_explode(self, n=0):
        if self.type == "integer":
            return False, None, None
        if n == 4:
            l, r = self.content[0], self.content[1]
            self.content = 0
            return True, l, r
        left = Number(self.content[0])
        right = Number(self.content[1])
        lexplode, lleft, lright = left.do_explode(n+1)
        if lexplode:
            self.content = [left.content, right.add_to_left(lright)]
            return True, lleft, None
        rexplode, rleft, rright = right.do_explode(n+1)
        if rexplode:
            self.content = [left.add_to_right(rleft), right.content]
            return True, None, rright
        return False, None, None
    
    def do_split(self):
        if self.type == "integer":
            value = self.content
            if value > 9:
                self.content = [value // 2, (value + 1) // 2]
                return True
            else:
                return False
        else:
            left = Number(self.content[0])
            right = Number(self.content[1])
            if left.do_split():
                self.content[0] = left.content
                return True
            elif right.do_split():
                self.content[1] = right.content
                return True
            else:
                return False
    
    def do_reduce(self, debug=False):
        if debug: print(" "* 12, self.content)
        while True:
            exploded, _, _ = self.do_explode()
            if debug: print("exploded".ljust(12), self.content)
            if not exploded:
                split = self.do_split()
                if debug: print("split".ljust(12), self.content)
                if not split:
                    return
    
    @property
    def magnitude(self):
        if self.type == "integer":
            return self.content
        left, right = Number(self.content[0]), Number(self.content[1])
        return 3 * left.magnitude + 2 * right.magnitude


In [159]:
def solve1(inp):
    current = Number(inp[0])
    for i in range(1, len(inp)):
        current += Number(inp[i])
    return current.magnitude

def solve2(inp):
    magnitudes = []
    for x1 in inp:
        for x2 in inp:
            if x1 != x2:
                s = Number(x1) + Number(x2)
                magnitudes.append(s.magnitude)
    return max(magnitudes)

In [156]:
print(solve1(inputs_1))

1137


In [157]:
print(solve1(inputs_2))

3488


In [158]:
solve1(inputs)

4391

In [160]:
solve2(inputs)

4626

In [ ]:
solve2()